In [11]:
from GapEstimator import BagProcedure, GapProblem
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm

# Define the CVaR problem

In [12]:
cvarAlpha = 0.1
def cvarSAA(data, x0=None):
#     This function solves the SAA problem given the data
    
#     data: data for uncertain parameters in the optimization problem
#     x0: a solution of the optimization problem; default None, solve the SAA with the modified objective h(x,\xi) - h(x0,\xi) instead if specified
    
#     return: a tuple of length 3: SAA solution, SAA objective value, SAA objective variance

    n = len(data)
    data = np.sort(data)
    idx = int(np.ceil(cvarAlpha * n))
    x = data[-idx]
    mean, var = cvarSAAObj(data, x, x0=x0)
    return x, mean, var

def cvarSAAObj(data, x, x0=None):
#     This function evaluates the SAA objective given the data and an arbitrary feasible solution
    
#     data: data for uncertain parameters in the optimization problem
#     x: an arbitrary feasible solution at which the SAA objective needs to be evaluated
#     x0: a solution of the optimization problem; default None, evaluate the SAA objective with the modified objective h(x,\xi) - h(x0,\xi) instead if specified
    
#     return: a tuple of length 2: SAA objective at x, SAA objective variance at x
    
    SAA_cost = x + np.maximum(data - x, 0) / cvarAlpha
    if x0 is not None:
        SAA_cost -= x0 + np.maximum(data - x0, 0) / cvarAlpha
    return np.mean(SAA_cost), np.var(SAA_cost)


def cvarObj(x):
    # to compute the exact objective value
    def func(t):
        return 1 / np.sqrt(2 * np.pi) * np.exp(-t**2 / 2) * (t - x)
    return x + quad(func, x, np.inf)[0] / cvarAlpha

gapProblem = GapProblem(cvarSAA, cvarSAAObj)

# Define the bagging estimator

In [13]:
# withReplace = False
withReplace = True
# resample with or without replacement

useDebias = True 
# whether or not debias the infinitesimal Jackknife variance estimator, True recommended

includePointVar = True 
# whether or not include variance of the bagging point estimator in the bound, True recommended

resample_rng = np.random.default_rng(seed = 123456)


bagEstimator = BagProcedure(replace = withReplace, debias = useDebias, pointVar = includePointVar, rng = resample_rng)

# Generate data

In [14]:
data_rng = np.random.default_rng(seed = 654321)

N = 200
# data size

k = 150
# resample size

B = 500
# bootstrap size

alpha = 0.05
# target confidence level = 1 - alpha

xstar = norm.ppf(1 - cvarAlpha)
# true optimal value

data = data_rng.normal(size = N)

# Compute lower bounds of the optimal value

In [15]:
bound, point_estimate, var_estimate = bagEstimator.run(gapProblem, data, alpha, k, B) # this returns lower bounds of the optimal value
print("Exact optimal value = ", cvarObj(xstar), "\nBagging bound = ", bound, "\npoint estimate = ", point_estimate, "\nvariance estimate = ", var_estimate)

Exact optimal value =  1.754983319324868 
Bagging bound =  1.6114000509818678 
point estimate =  1.7674183854112808 
variance estimate =  0.008996980122896124


# Compute upper bounds of optimality gap for the solution x = 1.5

In [16]:
x0 = 1.5

bound2, point_estimate2, var_estimate2 = bagEstimator.run(gapProblem, data, alpha, k, B, x0 = x0) # this returns upper bounds of the optimality gap for x0
print("Exact optimality gap = ", cvarObj(x0) - cvarObj(xstar), "\nBagging bound = ", bound2, "\npoint estimate = ", point_estimate2, "\nvariance estimate = ", var_estimate2)

Exact optimality gap =  0.038084618301178264 
Bagging bound =  0.057434597247277994 
point estimate =  0.026423804082452065 
variance estimate =  0.0003554440388883513
